In [ ]:
from zernike_algos import (
    ZernikePolynomial,
    zernike_radial,
    zernike_radial_optimized,
)
import numpy as np
import jax.numpy as jnp
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
basis = ZernikePolynomial(L=50, M=50, spectral_indexing="ansi", sym="cos")
r = np.linspace(0, 1, 100)
radial = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 0)
print(radial)
# print(radial)
# l = basis.modes[:,0]
# m = basis.modes[:,1]
# m = np.abs(m)
# n = (l-m)//2
# idx = np.lexsort((n,m))
# id0 = np.arange(0,len(l))
# id0 = id0[idx]
# l = l[idx]
# m = m[idx]
# n = n[idx]

# l_org = basis.modes[:,0]
# l0 = l[np.argsort(id0)]

# check = (l-m) % 2
# # Get the unique values in array1
# unique_values = np.unique(m)
# opt_param = []
# # For each unique value, find the maximum value in array2 where this value occurs in array1
# for value in unique_values:
#     indices = np.where(m == value)
#     max_value = np.max(n[indices])
#     opt_param.append(np.array([value, max_value]))
# opt_param = np.array(opt_param)
# m_opt = opt_param[:,0]
# n_opt = opt_param[:,1]
# check = np.zeros(m_opt.shape)

# radial_opt = zernike_radial_optimized(r[:,np.newaxis], check, m_opt, n_opt, 0)
# print(radial_opt)
# print(radial_opt.shape)
# radial_opt = radial_opt[:, np.argsort(id0)]
# print(radial-radial_opt)


In [ ]:
print("zernike_radial, 0th derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 0).block_until_ready()
print("zernike_radial, 1st derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 1).block_until_ready()
print("zernike_radial, 2nd derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 2).block_until_ready()
print("zernike_radial, 3rd derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 3).block_until_ready()

In [ ]:
import numpy as np

def jacobi_poly(x, n, alpha, beta):
    """
    Calculate the Jacobi polynomial P_n^(alpha, beta) at points x.

    Parameters:
    - x: array-like, Points where the Jacobi polynomial is evaluated.
    - n: array-like, Degree of the Jacobi polynomial.
    - alpha: array-like, Alpha parameter of the Jacobi polynomial.
    - beta: array-like, Beta parameter of the Jacobi polynomial.

    Returns:
    - P_n: array, Values of the Jacobi polynomial at points x.
    """
    result = []
    x = 1 - 2 * x**2
    for i in range(len(alpha)):
        P_n1 = jacobi_poly_single(x, 1, alpha[i], beta,)
        P_n2 = jacobi_poly_single(x, 0, alpha[i], beta,)
        power = x**alpha[i]
        result.append(np.array((-1) ** 0 * power * P_n2))
        if n[i] >= 1:
            result.append(np.array((-1) ** 1 * power * P_n1))
        for N in range(2, n[i] + 1):
            P_n = jacobi_poly_single(x, N, alpha[i], beta, P_n1, P_n2)
            result.append(np.array((-1) ** N * power * P_n))
            P_n2 = P_n1
            P_n1 = P_n
    return np.transpose(np.array(result))

def jacobi_poly_single(x, n, alpha, beta, P_n1=0, P_n2=0):
    if n == 0:
        return np.ones_like(x)
    elif n == 1:
        return 0.5 * ((alpha + beta + 2) + (alpha - beta) * (x - 1))
    else:
        c = 2*n + alpha + beta
        a1 = 2*n* (c-n) * (c-2)
        a2 = (c-1) * ( c*(c-2)*x + (alpha-beta)*(alpha+beta) )
        a3 = 2*(n+alpha-1)*(n+beta-1)*c
        
        P_n = (a2*P_n1 - a3*P_n2) / a1

        return np.where(type(n) == int, P_n, 0)

basis = ZernikePolynomial(L=50, M=50, spectral_indexing="ansi", sym="cos")
r = np.linspace(0, 1, 100)
# print(radial)
l = basis.modes[:,0]
m = basis.modes[:,1]
m = np.abs(m)
n = (l-m)//2

idx = np.lexsort((n,m))
id0 = np.arange(0,len(l))
id0 = id0[idx]

unique_values = np.unique(m)
opt_param = []
# For each unique value, find the maximum value in array2 where this value occurs in array1
for value in unique_values:
    indices = np.where(m == value)
    max_value = np.max(n[indices])
    opt_param.append(np.array([value, max_value]))
opt_param = np.array(opt_param)
m_opt = opt_param[:,0]
n_opt = opt_param[:,1]

# for i in range(len(n_opt)):
#     print(f"{m_opt[i]}    {n_opt[i]}")

# print(n)
# Broadcasting is used for element-wise operations
jacobi_values = jacobi_poly(r, n_opt, m_opt, 0)
jacobi_values = np.where((l - m) % 2 == 0, jacobi_values, 0)
print(jacobi_values.shape)
print(jacobi_values)


In [ ]:
print(jacobi_values-radial)